# Rayleigh numbers

In [1]:
import ipyparallel as ipp

N_PROC = 2
cluster = ipp.Cluster(engines='mpi', n=N_PROC)
rc = cluster.start_and_connect_sync()
view = rc.load_balanced_view()

Starting 2 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>
100%|██████████| 2/2 [00:09<00:00,  4.66s/engine]


In [ ]:
from typing import Callable
from functools import partial
from lucifex.sim import Simulation, GridSimulation
from lucifex.viz import plot_colormap, plot_line


def ipp_simulation(
    Ra: float,
    Nx: int,
    Ny: int,
    store: int,
    **run_kwargs,
) -> GridSimulation:
    from mpi4py import MPI
    from lucifex.solver import OptionsPETSc
    from lucifex.fdm import AB, CN
    from lucifex.sim import run
    from crocodil.dns import dns_system_a, SYSTEM_A_REFERENCE
    physical = dict(SYSTEM_A_REFERENCE)
    physical.update(Ra=Ra)
    sim = dns_system_a(
        store_delta=store, 
    )(
        comm=MPI.COMM_SELF,
        Nx=Nx,
        Ny=Ny,
        scaling='advective',
        **SYSTEM_A_REFERENCE,
        D_adv=AB(1)@CN,
        D_diff=AB(1)@CN,
        c_petsc=OptionsPETSc('gmres', 'ilu'),
        flow_petsc=(OptionsPETSc('cg', 'hypre'), None),
        diagnostic=True,
    )

    if run_kwargs:
        run(sim, **run_kwargs)

    return GridSimulation.from_simulation(sim)


STORE = 1
NX = 60
NY = 60

N_STOP = 10
T_STOP = 20.0
DT_INIT = 1e-6
N_INIT = 6

Ra_opts = (500.0, 800.0)

run_func = None
# run_func = partial(ipp_run, n_stop=N_STOP, t_stop=T_STOP, dt_init=DT_INIT, n_init=N_INIT)
sim_func = partial(
    ipp_simulation,
    Nx=NX,
    Ny=NY,
    store=STORE,
    run_func=run_func,
)
futures = view.map_async(
    partial(sim_func, run_func=run_func), Ra_opts,
)
simulations: dict[float, Simulation] = dict(
    zip(Ra_opts, futures.get()),
)


In [3]:
from mpi4py import MPI

from joblib import Parallel, delayed
from math import sqrt

from lucifex.mesh import rectangle_mesh
from lucifex.fem import Function


def job(Nx):
    comm = MPI.COMM_SELF
    mesh = rectangle_mesh(1.0, 1.0, Nx, Nx, comm=comm)
    f = Function((mesh, 'P', 1))
    dofs = f.x.array[:]
    return lambda: dofs


Nx_opts = (10, 20, 30)

Parallel(n_jobs=2)(
    delayed(job)(nx) for nx in Nx_opts
)

[<function __main__.job.<locals>.<lambda>()>,
 <function __main__.job.<locals>.<lambda>()>,
 <function __main__.job.<locals>.<lambda>()>]